In [2]:
from periomod.data import StaticProcessEngine
from periomod.descriptives import DescriptivesPlotter
from periomod.data import _helpers
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns

In [77]:
engine = StaticProcessEngine(behavior=False)
df = engine.load_data("/home/ewalter/pa-modeling/data/raw/Periodontitis_ML_Dataset.xlsx")
pdf = engine.process_data(df)
#plotter = DescriptivesPlotter(df)
#df.replace(" ", pd.NA)
df.columns

Initial number of patients: 746
Initial number of rows: 100950
Number of unique patients removed: 7
Number of rows removed: 1026
Remaining number of patients: 739
Remaining number of rows: 99924



/home/ewalter/pa-modeling/periomod/data/_preprocessing.py:275: UserWarning: Missing values found: 
bodymassindex              1560
periofamilyhistory        10056
diabetes                    666
smokingtype               38220
cigarettenumber           85164
stresslvl                  2502
boprevaluation              567
percussion-sensitivity      222
sensitivity                 312
furcationbaseline         88304
recbaseline                9878
plaque                     2513
bop                         633
dtype: int64
  self.create_tooth_features(self.impute_missing_values(data=data))


No missing values after imputation.


Index(['id_patient', 'age', 'gender', 'bodymassindex', 'periofamilyhistory',
       'diabetes', 'smokingtype', 'cigarettenumber', 'antibiotictreatment',
       'stresslvl', 'pdrevaluation', 'boprevaluation', 'pregnant', 'tooth',
       'toothtype', 'rootnumber', 'mobility', 'restoration',
       'percussion-sensitivity', 'sensitivity', 'furcationbaseline', 'side',
       'pdbaseline', 'recbaseline', 'plaque', 'bop'],
      dtype='object')

In [67]:
Patientcategorialvariables=["Gender", "PerioFamilyHistory", "Diabetes", "SmokingType", "AntibioticTreatment"]
patientcategorial=patientdf[Patientcategorialvariables]

smokipdf=patientdf[patientdf["SmokingType"]>1]
smokingnumber=smokipdf["CigaretteNumber"]

patientdf["Stresslvl"].median()

5.0

In [ ]:
def categorical_distribution(df):
    result = []
    total_patients = df.shape[0]  # Maximum number of entries per feature

    for col in df.columns:
        value_counts = df[col].value_counts(dropna=False)  # Count all, including NaN
        for category, count in value_counts.items():
            percentage = (count / total_patients) * 100  # Percentage calculation
            result.append(["Patient", col, category, count, total_patients, percentage])

    summary_df = pd.DataFrame(result, columns=["Level", "Feature", "Category", "Count", "Total", "Percentage"])
    return summary_df

patientdistr= categorical_distribution(patientcategorial)

print(patientdistr.to_latex(index=False))



\begin{tabular}{llrrrr}
\toprule
Level & Feature & Category & Count & Total & Percentage \\
\midrule
Patient & Gender & 2.000000 & 393 & 746 & 52.680965 \\
Patient & Gender & 1.000000 & 348 & 746 & 46.648794 \\
Patient & Gender & 0.000000 & 5 & 746 & 0.670241 \\
Patient & PerioFamilyHistory & 2.000000 & 246 & 746 & 32.975871 \\
Patient & PerioFamilyHistory & 1.000000 & 236 & 746 & 31.635389 \\
Patient & PerioFamilyHistory & 3.000000 & 186 & 746 & 24.932976 \\
Patient & PerioFamilyHistory & NaN & 78 & 746 & 10.455764 \\
Patient & Diabetes & 1.000000 & 670 & 746 & 89.812332 \\
Patient & Diabetes & 3.000000 & 31 & 746 & 4.155496 \\
Patient & Diabetes & 4.000000 & 26 & 746 & 3.485255 \\
Patient & Diabetes & 2.000000 & 13 & 746 & 1.742627 \\
Patient & Diabetes & NaN & 6 & 746 & 0.804290 \\
Patient & SmokingType & 1.000000 & 339 & 746 & 45.442359 \\
Patient & SmokingType & NaN & 280 & 746 & 37.533512 \\
Patient & SmokingType & 2.000000 & 113 & 746 & 15.147453 \\
Patient & SmokingType & 3.000

In [ ]:
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [76]:
patientdf=df.groupby("ID_patient").mean()
toothdfgrouped = df.groupby(["ID_patient", "Tooth"]).mean()

toothdfgrouped.isna().sum()
patientdf.isna().sum()
df.isna().sum()

patientdf["SmokingType"].value_counts()
patientdf["Stresslvl"]


KeyError: 'ID_patient'

In [ ]:
#This filteres out all rows that can have a furcation involvement
sides=_helpers._get_side()
furcationdf= df["FurcationBaseline"]
condition = df.apply(lambda row: any(row["Tooth"] in teeth and row["Side"] in sides for teeth, sides in sides.items()), axis=1)
filtered_df = df[condition]
empty_count = filtered_df.isna().sum()
print(empty_count)

df.describe()


ID_patient                    0
Age                           0
Gender                        0
BodyMassIndex               228
PerioFamilyHistory         1439
Diabetes                    119
SmokingType                4975
CigaretteNumber           11027
AntibioticTreatment           0
Stresslvl                   374
PdRevaluation                 0
BOPRevaluation               71
Pregnant                   5864
Tooth                         0
Toothtype                     0
RootNumber                    0
Mobility                      0
Restoration                   0
Percussion-sensitivity       28
Sensitivity                  51
FurcationBaseline          1086
Side                          0
PdBaseline                    0
RecBaseline                1234
Plaque                      391
BOP                          91
dtype: int64


,ID_patient,Age,Gender,AntibioticTreatment,PdRevaluation,BOPRevaluation,Tooth,Toothtype,RootNumber,Mobility,Restoration,Side,PdBaseline
count,100950.000000,100950.000000,100950.000000,100950.000000,100950.000000,100383.000000,100950.000000,100950.000000,100950.000000,100950.000000,100950.000000,100950.000000,100950.000000
mean,778.084042,56.269718,1.521783,1.856701,2.567132,1.279320,29.300802,1.784903,1.319762,1.840357,1.756077,3.500000,2.818415
std,550.163142,14.021206,0.512565,0.350379,1.222458,0.448667,11.385683,0.823888,0.466387,0.366277,0.812717,1.707834,1.407334
min,1.000000,15.000000,0.000000,1.000000,1.000000,1.000000,11.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,326.000000,48.000000,1.000000,2.000000,2.000000,1.000000,21.000000,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000
50%,655.000000,58.000000,2.000000,2.000000,2.000000,1.000000,31.000000,2.000000,1.000000,2.000000,2.000000,3.500000,3.000000
75%,1203.000000,67.000000,2.000000,2.000000,3.000000,2.000000,41.000000,3.000000,2.000000,2.000000,2.000000,5.000000,3.000000
max,1948.000000,88.000000,2.000000,2.000000,12.000000,2.000000,48.000000,3.000000,2.000000,2.000000,3.000000,6.000000,12.000000


In [ ]:
plotter.histogram_2d("pdbaseline", "pdrevaluation")

In [ ]:
plotter.pocket_comparison("pdbaseline", "pdrevaluation")

In [ ]:
plotter.pocket_group_comparison("pdgroupbase", "pdgrouprevaluation")

In [ ]:
plotter.plt_matrix("pdgrouprevaluation", "pdgroupbase")

In [ ]:
plotter.outcome_descriptive("pdgrouprevaluation", "Distribution of Classes")